# 第5讲：相机与图像

视觉SLAM中，观测主要是指**相机成像**的过程。


## 5.1 相机模型

相机将三维世界中的坐标点（米）映射到二维图像平面（像素）的过程能够用几何模型描述，其中最简单模型为**针孔模型**。针孔模型描述了一束光线通过针孔之后，在针孔背面投影成像的关系。但由于相机镜头的透镜作用，光线投影到成像平面的过程中会产生**畸变**。因此，相机投影成像过程需要使用针孔和畸变两个模型来描述，这两个模型能够把外部三维点投影到相机内部成像平面上，构成相机的**内参数**。

### 5.1.1 针孔相机模型

设$O−x−y−z$为相机坐标系，通常$z$轴指向相机前方、$x$向右、$y$向下。$O$为摄像机光心（针孔模型中的针孔）。点$P$经过小孔$O$投影后，落在物理成像平面$O^{\prime}-x^{\prime}-y^{\prime}$上，成像点为$P^{\prime}$。

<img src="./img/fig_5_1.png" width="500" />

在相机坐标系下，设$P$点坐标为$[X, Y, Z]^{\text{T}}$，$P^{\prime}$为$[X^{\prime}, Y^{\prime}, Z^{\prime}]^{\text{T}}$，并且设物理成像平面到小孔的距离为$f$（焦距）。根据三角形相似关系，

$$\frac{Z}{f} = - \frac{X}{X^{\prime}} = - \frac{Y}{Y^{\prime}} \tag {5-1}$$

其中，负号表示成像是倒立的。为简化模型，将成像平面对称到相机前方，在摄像机坐标系中，位于点$P$同侧。此进，可将负号去掉：

$$\frac{Z}{f} = \frac{X}{X^{\prime}} = \frac{Y}{Y^{\prime}} \tag {5-2}$$

整理得：

$$\begin{aligned}
X^{\prime} & = f \frac{X}{Z} \\
Y^{\prime} & = f \frac{Y}{Z} \\
\end{aligned} \tag {5-3}$$


<img src="./img/fig_5_2.png" width="600" />

方程（5-3）表示点$P$与其像之间的空间关系。在相机中，观测数据为像素，因此需要在成像平面上对像进行采样和量化。假设物理成像平面上固定一个像素平面$o − u − v$，在像素平面得到了$P^{\prime}$的像素坐标：$[u, v]^{\text{T}}$。

**像素坐标系（图像坐标系）**的定义为：原点$o^{\prime}$位于图像的左上角，$u$轴向右与$x$轴平行，$v$轴向下与$y$轴平行。像素坐标系与成像平面之间，相差一个缩放和一个原点的平移。假设像素坐标在$u$轴上缩放了$\alpha$倍，在$v$上缩放了$\beta$倍，同时原点平移$[c_{x}, c_{y}]^{\text{T}}$，则$P^{\prime}$的坐标与像素坐标$[u, v]^{\text{T}}$的关系为：

$$\begin{cases}
u = \alpha X^{\prime} + c_{x} \\
v = \alpha Y^{\prime} + c_{y} \\
\end{cases} \tag {5-4}$$

代入方程（5-3）可得

$$\begin{cases}
u = f_{x} \frac{X}{Z} + c_{x} \\
v = f_{y} \frac{Y}{Z} + c_{y} \\
\end{cases} \tag {5-5}$$

其中，$f$的单位为米，$\alpha$、$\beta$单位为像素每米，$f_{x}$、$f_{x}$的单位为像素。使用齐次坐标，将该式写为矩阵形式：

$$\begin{bmatrix}
    u \\ v \\ 1
\end{bmatrix} = \frac{1}{Z} \begin{bmatrix}
    f_{x} & 0 & c_{x} \\
    0 & f_{y} & c_{y} \\
    0 & 0 & 1
\end{bmatrix} \begin{bmatrix}
    X \\ Y \\ Z
\end{bmatrix} \triangleq
\frac{1}{Z} \mathbf{K} \mathbf{P} \tag {5-6}$$

$$Z \begin{bmatrix}
    u \\ v \\ 1
\end{bmatrix} = \begin{bmatrix}
    f_{x} & 0 & c_{x} \\
    0 & f_{y} & c_{y} \\
    0 & 0 & 1
\end{bmatrix} \begin{bmatrix}
    X \\ Y \\ Z
\end{bmatrix} \triangleq
\mathbf{K} \mathbf{P} \tag {5-7}$$

其中，矩阵$\mathbf{K}$称为相机的内参数矩阵（camera intrinsics）。**标定**：确定相机的内参。

相机外参（camera extrinsics）指相机的位姿$\mathbf{R}$、$\mathbf{t}$。假设点$P$的世界坐标记为$P_{w}$，其相机坐标是根据相机的位姿，变换到相机坐标系下的结果：

$$Z \mathbf{P}_{uv} = Z \begin{bmatrix}
    u \\ v \\ 1
\end{bmatrix} = \mathbf{K} (\mathbf{R} \mathbf{P}_{w} + \mathbf{t}) = \mathbf{K} \mathbf{T} \mathbf{P}_{w} \tag {5-8}$$

其中，$\mathbf{T}$为欧氏变换矩阵（$\mathbf{K} \mathbf{T} \mathbf{P}_{w}$为齐次坐标），方程（5-8）表示点$P$的世界坐标到像素坐标的投影。相机外参随相机的运动而改变，是SLAM中待估计目标，代表着机器人的轨迹。根据齐次坐标性质，方程（5-8）可简化为：

$$\mathbf{P}_{uv} = \mathbf{K} \mathbf{T} \mathbf{P}_{w} \tag {5-9}$$



### 5.1.2 畸变

<img src="./img/fig_5_3.png" width="500" />
